In [1]:
pip install datapackage

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
from datapackage import Package

package = Package('https://datahub.io/sports-data/spanish-la-liga/datapackage.json')

columns_package = ['Div','Date','HomeTeam','AwayTeam','FTHG','FTAG','FTR','HTHG','HTAG','HTR','HS','AS','HST','AST','HF','AF','HC','AC','HY','AY','HR','AR','B365H','B365D','B365A']

dictionary_variables = {'Div':'League Division','Date':'Match Date (dd/mm/yy)','Time':'Match Time','HomeTeam':'Home Team','AwayTeam':'Away Team','FTHG':'Full Time Home Team Goals','FTAG':'Full Time Away Team Goals','FTR':'Full Time Result (H Home Win, D Draw, A Away Win)','HS':'Home Team Shots','AS':'Away Team Shots','HST':'Home Team Shots on Target','AST':'Away Team Shots on Target','B365H':'Bet365 home win odds','B365D':'Bet365 draw odds','B365A':'Bet365 away win odds','season':'number of season'}

# Create historical data : season 2009-2010 to 2018-2019

historical_data = pd.DataFrame()

for resource in package.resources:
        if resource.descriptor['datahub']['type'] == 'derived/csv':
            x = pd.DataFrame(data = resource.read())
            x = x.drop(x.columns[range(25,len(x.columns))], axis=1)
            x.columns = columns_package
            x = x.drop(['HTHG','HTAG','HTR','HF','AF','HC','AC','HY','AY','HR','AR'], axis = 1)
            x['season'] = resource.name[7:11]
            historical_data = historical_data.append(x) 

In [3]:
# Append season 2019-2020

season_1920 = pd.read_csv('https://www.football-data.co.uk/mmz4281/1920/SP1.csv', header = 0, usecols = ['Div', 'Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HS', 'AS', 'HST', 'AST', 'B365H', 'B365D', 'B365A'])
season_1920['season'] = '1920'
historical_data = historical_data.append(season_1920)

In [4]:
# Current season (2020-2021)

season_2021 = pd.read_csv('http://www.football-data.co.uk/mmz4281/2021/SP1.csv', header = 0, usecols = ['Div', 'Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HS', 'AS', 'HST', 'AST', 'B365H', 'B365D', 'B365A'])
season_2021['season'] = '2021'
historical_data = historical_data.append(season_2021)

In [5]:
# Fitting some format variables

historical_data['Date'] = pd.to_datetime(historical_data['Date'], infer_datetime_format=True)
historical_data['B365H'] = historical_data['B365H'].astype(float)
historical_data['B365D'] = historical_data['B365D'].astype(float)
historical_data['B365A'] = historical_data['B365A'].astype(float)

In [6]:
historical_data.head()

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HS,AS,HST,AST,B365H,B365D,B365A,season
0,SP1,2018-08-17,Betis,Levante,0,3,A,22,6,8,4,1.66,4.0,5.0,1819
1,SP1,2018-08-17,Girona,Valladolid,0,0,D,13,2,1,1,1.75,3.6,5.0,1819
2,SP1,2018-08-18,Barcelona,Alaves,3,0,H,25,3,9,0,1.11,10.0,21.0,1819
3,SP1,2018-08-18,Celta,Espanol,1,1,D,12,14,2,5,1.85,3.5,4.5,1819
4,SP1,2018-08-18,Villarreal,Sociedad,1,2,A,16,8,7,4,2.04,3.4,3.8,1819


I keep the variables: goals, shoots and BET365 quotas for coming purposes but for the final database can not be included ( because would be flawness estimators)

In [7]:
# Headers
dictionary_variables

{'Div': 'League Division',
 'Date': 'Match Date (dd/mm/yy)',
 'Time': 'Match Time',
 'HomeTeam': 'Home Team',
 'AwayTeam': 'Away Team',
 'FTHG': 'Full Time Home Team Goals',
 'FTAG': 'Full Time Away Team Goals',
 'FTR': 'Full Time Result (H Home Win, D Draw, A Away Win)',
 'HS': 'Home Team Shots',
 'AS': 'Away Team Shots',
 'HST': 'Home Team Shots on Target',
 'AST': 'Away Team Shots on Target',
 'B365H': 'Bet365 home win odds',
 'B365D': 'Bet365 draw odds',
 'B365A': 'Bet365 away win odds',
 'season': 'number of season'}

In [8]:
historical_data.head()

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HS,AS,HST,AST,B365H,B365D,B365A,season
0,SP1,2018-08-17,Betis,Levante,0,3,A,22,6,8,4,1.66,4.0,5.0,1819
1,SP1,2018-08-17,Girona,Valladolid,0,0,D,13,2,1,1,1.75,3.6,5.0,1819
2,SP1,2018-08-18,Barcelona,Alaves,3,0,H,25,3,9,0,1.11,10.0,21.0,1819
3,SP1,2018-08-18,Celta,Espanol,1,1,D,12,14,2,5,1.85,3.5,4.5,1819
4,SP1,2018-08-18,Villarreal,Sociedad,1,2,A,16,8,7,4,2.04,3.4,3.8,1819


Now a clasification by season/team and date by date is needed to calculate the trends in terms of points, winning/lossing streaks, goals, shoots,etc.

In [9]:
# Table by season|HomeTeam|Date

aux_table_1 = pd.DataFrame(historical_data.groupby(['season','HomeTeam', 'Date'])['Div'].count()).reset_index()
aux_table_1.drop(columns =['Div'], inplace=True)
aux_table_1.rename(columns={"HomeTeam": "Team"}, inplace = True)
aux_table_1['Home/Away'] = 'H'

# Table by season|AwayTeam|Date
aux_table_2 = pd.DataFrame(historical_data.groupby(['season','AwayTeam', 'Date'])['Div'].count()).reset_index()
aux_table_2.drop(columns =['Div'], inplace=True)
aux_table_2.rename(columns={"AwayTeam": "Team"}, inplace = True)
aux_table_2['Home/Away'] = 'A'

# Table by season|Team|Date
aux_table = aux_table_1.append(aux_table_2, ignore_index=True, sort = True)
aux_table.sort_values(by=['season','Team','Date'] ,ascending = True, ignore_index=False, inplace = True)
aux_table.reset_index(inplace=True)
aux_table.drop(columns =['index'], inplace=True)

# Assigning number of match
aux_table['match']  = aux_table.groupby(['season', 'Team']).cumcount() + 1

In [10]:
aux_table

,Date,Home/Away,Team,season,match
0,2009-08-30,H,Almeria,0910,1
1,2009-09-13,A,Almeria,0910,2
2,2009-09-20,H,Almeria,0910,3
3,2009-09-23,A,Almeria,0910,4
4,2009-09-27,H,Almeria,0910,5
...,...,...,...,...,...
8935,2021-03-14,A,Villarreal,2021,25
8936,2021-03-21,H,Villarreal,2021,26
8937,2021-05-03,A,Villarreal,2021,27
8938,2021-06-02,A,Villarreal,2021,28


Now I will add the information contained in the historica dataset

In [11]:
# Adding the variables from historical dataset

historical_acc_1 = aux_table.merge(historical_data, how = 'inner', left_on=['Date','Team'], right_on=['Date','HomeTeam'])
historical_acc_2 = aux_table.merge(historical_data, how = 'inner', left_on=['Date','Team'], right_on=['Date','AwayTeam'])
historical_acc = historical_acc_1.append(historical_acc_2)

# Rearranging

historical_acc.sort_values(by=['season_x','Team','Date'], inplace = True)

# Calculation of the points

historical_acc['victory']  = historical_acc['Home/Away'] == historical_acc['FTR'] 
historical_acc['draw'] = historical_acc['FTR'] == 'D'
historical_acc['points'] = (historical_acc['victory']) * 3 + historical_acc['draw']
                           
# Removing useless columns
                           
historical_acc.drop(labels=['HomeTeam','AwayTeam','season_y','victory','draw'], axis = 1, inplace = True)
historical_acc.rename(columns = {'season_x': 'season'}, inplace = False)

,Date,Home/Away,Team,season,match,Div,FTHG,FTAG,FTR,HS,AS,HST,AST,B365H,B365D,B365A,points
0,2009-08-30,H,Almeria,0910,1,SP1,0,0,D,20,7,5,1,2.10,3.30,3.50,1
0,2009-09-13,A,Almeria,0910,2,SP1,1,0,H,16,7,4,0,2.38,3.25,3.00,0
1,2009-09-20,H,Almeria,0910,3,SP1,1,0,H,11,23,3,11,2.50,3.25,2.80,3
1,2009-09-23,A,Almeria,0910,4,SP1,2,2,D,24,12,9,7,1.44,4.33,7.00,1
2,2009-09-27,H,Almeria,0910,5,SP1,2,2,D,13,13,4,6,2.25,3.25,3.20,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4466,2021-03-14,A,Villarreal,2021,25,SP1,1,3,A,16,13,7,8,2.90,3.40,2.40,3
4469,2021-03-21,H,Villarreal,2021,26,SP1,2,1,H,13,9,4,3,1.44,4.20,7.50,3
4467,2021-05-03,A,Villarreal,2021,27,SP1,2,1,H,25,10,4,5,4.00,3.40,1.95,0
4468,2021-06-02,A,Villarreal,2021,28,SP1,2,2,D,8,9,2,4,6.50,3.80,1.53,1


In the same way I calculated the points I will calculate the Goals/Shots/Shots in Target

In [12]:
# Calculation of the goals/shots/shots on target

historical_acc['in_home']  = historical_acc['Home/Away'] == 'H'
historical_acc['away_game']  = historical_acc['Home/Away'] == 'A'

# For

historical_acc['goals_for'] = (historical_acc['in_home'] * historical_acc['FTHG']) + (historical_acc['away_game'] * historical_acc['FTAG'])
historical_acc['shots_for'] = (historical_acc['in_home'] * historical_acc['HS']) + (historical_acc['away_game'] * historical_acc['AS'])
historical_acc['shots_target_for'] = (historical_acc['in_home'] * historical_acc['HST']) + (historical_acc['away_game'] * historical_acc['AST'])

# Against

historical_acc['goals_against'] = (historical_acc['in_home'] * historical_acc['FTAG']) + (historical_acc['away_game'] * historical_acc['FTHG'])
historical_acc['shots_against'] = (historical_acc['in_home'] * historical_acc['AS']) + (historical_acc['away_game'] * historical_acc['HS'])
historical_acc['shots_target_against'] = (historical_acc['in_home'] * historical_acc['AST']) + (historical_acc['away_game'] * historical_acc['HST'])


historical_acc.reset_index(inplace = True)
historical_acc.drop(columns=['index','in_home','away_game'], inplace = True)

In [13]:
historical_acc.head(40)

,Date,Home/Away,Team,season_x,match,Div,FTHG,FTAG,FTR,HS,AS,HST,AST,points,goals_for,shots_for,shots_target_for,goals_against,shots_against,shots_target_against
0,2009-08-30,H,Almeria,0910,1,SP1,0,0,D,20,7,5,1,1,0,20,5,0,7,1
1,2009-09-13,A,Almeria,0910,2,SP1,1,0,H,16,7,4,0,0,0,7,0,1,16,4
2,2009-09-20,H,Almeria,0910,3,SP1,1,0,H,11,23,3,11,3,1,11,3,0,23,11
3,2009-09-23,A,Almeria,0910,4,SP1,2,2,D,24,12,9,7,1,2,12,7,2,24,9
4,2009-09-27,H,Almeria,0910,5,SP1,2,2,D,13,13,4,6,1,2,13,4,2,13,6
5,2009-10-03,A,Almeria,0910,6,SP1,1,0,H,17,0,7,0,0,0,0,0,1,17,7
6,2009-10-18,A,Almeria,0910,7,SP1,1,2,A,16,18,5,7,3,2,18,7,1,16,5
7,2009-10-25,H,Almeria,0910,8,SP1,0,3,A,9,18,2,7,0,0,9,2,3,18,7
8,2009-11-01,A,Almeria,0910,9,SP1,2,1,H,21,12,6,4,0,1,12,4,2,21,6
9,2009-11-08,H,Almeria,0910,10,SP1,2,0,H,10,10,3,1,3,2,10,3,0,10,1


In [32]:
dictionary_variables

{'Div': 'League Division',
 'Date': 'Match Date (dd/mm/yy)',
 'Time': 'Match Time',
 'HomeTeam': 'Home Team',
 'AwayTeam': 'Away Team',
 'FTHG': 'Full Time Home Team Goals',
 'FTAG': 'Full Time Away Team Goals',
 'FTR': 'Full Time Result (H Home Win, D Draw, A Away Win)',
 'HS': 'Home Team Shots',
 'AS': 'Away Team Shots',
 'HST': 'Home Team Shots on Target',
 'AST': 'Away Team Shots on Target',
 'B365H': 'Bet365 home win odds',
 'B365D': 'Bet365 draw odds',
 'B365A': 'Bet365 away win odds',
 'season': 'number of season'}